# [Project Report] 러닝 앱 유저 이탈 방어 전략: 행동 데이터 분석을 통한 골든타임 포착

- data : [Endomondo Fitness Trajectories](https://www.kaggle.com/datasets/pypiahmad/endomondo-fitness-trajectories)
## 1. 서론: 문제 정의 및 분석 배경 (Why?)
* **현상:** 러닝 붐으로 앱 신규 가입자는 늘었으나, **'작심삼일' 이탈률(D+30 Churn Rate)**이 60%에 육박함.
* **문제 의식 (Thinking Flow):**
    1. "사람들이 왜 그만둘까? 단순히 의지가 약해서일까?"
    2. "혹시 초보자가 자기 능력을 모르고 무리하게 뛰다가(Over-pacing) 부상을 입거나 지쳐서 그만두는 건 아닐까?"
    3. "숙련자는 정체기(Plateau)가 왔을 때 재미를 잃고 이탈하지 않을까?"
* **프로젝트 목표:** 유저의 **숙련도(Level)**에 따른 **이탈 원인(Trigger)**을 규명하고, 이탈 위험을 사전에 예측하는 모델 개발.

## 2. 분석 시나리오 및 가설 설정 (Hypothesis)
모델링 전에 반드시 확인해야 할 3가지 핵심 질문입니다.

### [Step 1] 유저 유형 정의 (Clustering)
> *"모든 유저를 똑같이 취급하면 안 된다. 누구는 '걷는 사람'이고 누구는 '마라토너'다."*
* **분석 방법:** `평균 속도`, `주당 빈도`, `1회 평균 거리`를 기준으로 K-Means Clustering 수행.
* **예상 그룹:**
    * **Group A (병아리):** 1km 미만, 불규칙적.
    * **Group B (열정맨):** 기록 향상이 빠르지만 심박수가 매우 높음 (위험군).
    * **Group C (고인물):** 페이스가 일정하고 기록 변화가 거의 없음.

### [Step 2] 유형별 이탈 원인 파악 (Deep Dive)
> *"열정맨과 고인물의 이탈 이유는 다르다."*
* **가설 1 (초보자):** **"오버페이스가 이탈을 부른다."**
    * 검증 지표: `심박수 대비 페이스 효율(Efficiency)`이 급격히 떨어지는 날 이후 7일 내 접속 여부 확인.
    * 논리: 능력치보다 무리하게 뛰면 신체적 고통으로 이어져 앱을 켜기 싫어짐.
* **가설 2 (숙련자):** **"기록 정체(Plateau)가 이탈을 부른다."**
    * 검증 지표: 최근 3개월간 기록(속도)의 `Standard Deviation(표준편차)`가 0에 수렴할 때 이탈률 변화.
    * 논리: 성취감이 사라지면 동기 부여가 떨어짐.

### [Step 3] 외부 요인과의 상관관계
* **가설 3:** "특정 계절이나 날씨가 이탈의 트리거가 된다."
    * 비가 3일 연속 온 뒤에 영구적으로 돌아오지 않는 유저 비율 분석.

## 3. 데이터 엔지니어링 (Feature Engineering)
위의 가설을 모델에 태우기 위해 구체적인 숫자로 변환하는 과정입니다.

| 변수명 | 산출 로직 | 의미 |
| :--- | :--- | :--- |
| **Overpacing_Count** | (내 평균 심박수 + 2σ)를 초과한 러닝 횟수 | "얼마나 자주 무리했는가?" |
| **Recovery_Time** | 이전 러닝과 현재 러닝 사이의 휴식 시간(Hour) | "충분히 쉬고 뛰는가?" (부상 위험) |
| **Performance_Slope** | 최근 5회 러닝 페이스의 선형 회귀 기울기 | "기록이 좋아지는 중인가, 나빠지는 중인가?" |
| **Consistency_Score** | 요일별 러닝 규칙성 (Entropy 활용) | "루틴이 깨지고 있는가?" |

## 4. 모델링 및 결과 도출 (Modeling)
* **알고리즘:** XGBoost (Feature Importance 확인 용이)
* **학습 목표:** T 시점의 데이터를 보고 T+14일 내 이탈 여부(Binary Classification) 예측.
* **기대 결과(Insight):**
    * "초보자 그룹(A)에서는 **'심박수 급증'** 변수가 이탈에 가장 큰 영향을 미쳤습니다."
    * "숙련자 그룹(C)에서는 **'러닝 빈도 감소'**가 가장 강력한 이탈 신호였습니다."

## 5. 솔루션 제안 (Streamlit Implementation)
분석 결과를 바탕으로 앱(Web)에서 어떤 액션을 취할 것인가?

* **기능 1: 러닝 MBTI 진단**
    * 유저 데이터를 넣으면 "당신은 **'불나방(Group B)'** 유형입니다. 초반에 너무 달려서 3주 안에 지칠 확률 90%!" 라고 진단.
* **기능 2: 골든타임 알림**
    * "데이터 분석 결과, 회원님은 지금 **'오버페이스'** 구간입니다. 오늘 속도를 10% 늦추지 않으면 내일 근육통으로 쉴 확률이 높습니다." (데이터 기반 조언)

# 📂 [프로젝트 정의서] 직업훈련 이탈 예측 모델 (Vocational Training Churn Prediction)

## 1. 데이터 개요 (Data Dictionary)
본 데이터셋은 대졸자직업이동경로조사(GOMS)를 기반으로, **직업훈련 수강생의 특성(X)**과 **중도 포기 여부(Y)**를 분석하기 위해 가공된 데이터입니다.

### 🎯 Target (예측 목표)
| 변수명 | 원본 코드 | 설명 | 값의 의미 |
| :--- | :--- | :--- | :--- |
| **Target_Dropout** | `g191l009` | **중도 포기 여부** | • **1**: 훈련 도중 포기 (이탈)<br>• **0**: 끝까지 수료 (성공) |

### 👤 학생 특성 (Student Specs)
| 변수명 | 설명 | 분석 가설 및 의미 |
| :--- | :--- | :--- |
| **Major_Category** | **전공 계열** | (1:인문 ~ 7:예체능) 비전공자(예: 문과)가 기술 교육(IT/공학)을 들을 때 이탈률이 높은가? |
| **GPA** | **대학 학점** | (4.5 만점) 성실성의 척도. 학점이 높은 학생이 훈련 수료율도 높은지 검증. |
| **Cert_Count** | **자격증 수** | 이미 보유한 자격증이 많을수록 취업 의지가 강해 이탈하지 않을 것이다. |
| **Uni_Type** | **학교 유형** | (2년제/4년제) 학교 학제에 따른 훈련 적응도 차이 분석. |
| **Sex** | **성별** | 성별에 따른 훈련 지속성 차이. |
| **Age** | **나이** | 나이가 많을수록 취업 절박함으로 인해 중도 포기율이 낮을 것이다. |

### 🏫 훈련 특성 (Training Features) ★ 핵심 변수
| 변수명 | 설명 | 분석 가설 및 의미 |
| :--- | :--- | :--- |
| **Training_Field** | **훈련 분야** | (NCS 코드) 요리/미용 과정 대비 **IT/SW 개발 과정**의 이탈률 비교. |
| **Training_Hours** | **총 훈련 시간** | 훈련 기간이 너무 길면(장기 과정) 지쳐서 그만두는 비율이 급증하는가? |
| **Self_Cost** | **본인 부담금** | **"무료 교육의 함정"** 검증.<br>내 돈(비용)을 지불한 학생은 아까워서라도 끝까지 버티는 경향이 있는가? |
| **Training_Helpfulness** | **수업 만족도** | 수업 만족도(1~5점)가 낮을 때 즉각적인 이탈로 이어지는지 확인. |

### 🏠 환경 변수 (Environment)
| 변수명 | 설명 | 분석 가설 및 의미 |
| :--- | :--- | :--- |
| **Family_Support** | **경제적 지원** | 부모님/가족의 경제적 지원(용돈 등)이 있을 때 학습에 더 집중하는가, 아니면 절박함이 떨어지는가? |

---

## 🚀 2. 분석 로드맵 (Analysis Roadmap)

### Phase 1. 데이터 탐색 (EDA): "패턴 발견하기"
모델링 전, 아래 3가지 가설을 시각화(그래프)로 검증하여 리포트에 포함합니다.
1.  **Money Effect:** `Self_Cost`(본인 부담금) 유무에 따른 이탈률 차이 막대그래프.
2.  **Mismatch Risk:** `Major`(인문계) 학생이 `Field`(공학) 수업을 들을 때의 이탈률 히트맵.
3.  **Burn-out Zone:** `Training_Hours`(훈련 시간) 증가에 따른 이탈률 변화 추이선.

### Phase 2. 예측 모델링 (Modeling)
* **알고리즘:** `RandomForest` 또는 `XGBoost` (설명 가능한 AI).
* **목표:** 단순 분류(0/1)를 넘어 **"이탈 확률(Probability)"**을 예측.
    * *예: "이 학생의 중도 포기 위험도는 82%입니다."*
* **Feature Importance:** 이탈에 가장 큰 영향을 미치는 요인이 무엇인지(예: 비용, 전공 등) 추출.

### Phase 3. 서비스 적용 시나리오 (Application)
**"Work24 훈련 과정 추천 시스템"**에 적용한다고 가정.

* **Scenario:** 문과생(학점 3.0)이 "6개월 무료 자바 개발자 과정" 클릭 시.
* **Action:** AI 모델이 위험도를 감지하여 경고 메시지 팝업.
    > 🚨 **"잠시만요! 이 과정은 회원님과 비슷한 스펙의 수강생 중 60%가 중도 포기했습니다."**
    > **💡 추천:** "완주율 95%인 **1개월 기초 파이썬 과정**부터 시작해보시는 건 어떨까요?"

In [1]:
import pandas as pd
df = pd.read_csv("GOMS.csv")
df.head()

,Major_Category,GPA,Training_Field,Training_Hours,Self_Cost,Family_Support,Cert_Count,Uni_Type,Sex,Age,Target_Dropout
0,1,3.80,14.0,792.0,0.0,300.0,7.0,1,2,26.333333,1
1,1,4.00,11.0,30.0,30.0,0.0,1.0,1,2,24.333333,0
2,1,3.70,11.0,35.0,-1.0,870.0,2.0,1,2,28.500000,0
3,1,2.03,24.0,3.0,-1.0,0.0,5.0,1,2,23.500000,0
4,1,3.00,11.0,100.0,30.0,200.0,1.0,1,2,24.750000,0
